In [ ]:
!pip install ta
!pip install pandas-ta
!pip install investpy

In [2]:
import pandas as pd
import numpy as np
import investpy
import seaborn as sns
from copy import copy
import matplotlib.pyplot as plt
import statistics as stats
import math
from ta.volume import MFIIndicator

# Import données #

In [3]:
df_source = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='29/04/2022')

df_source.drop(columns=['Currency','Exchange'], inplace=True)
df_source.reset_index(inplace=True)
df_source.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [16]:
df_source.head(1)

,Date,Open,High,Low,Close,Volume
0,2006-10-02,17.63,17.63,17.44,17.44,590


In [4]:
df_source.tail(1)

,Date,Open,High,Low,Close,Volume
3971,2022-04-29,653.3,660.0,633.6,639.0,7337


## Ajout indicateurs techniques ##

In [5]:
df = df_source.copy()

In [6]:
for i in range(5,51):
  s_mfi = MFIIndicator(high=df_source.High, low=df_source.Low, close=df_source.Close, volume=df_source.Volume, window=i).money_flow_index()
  df['MFI_'+str(i)] = round(s_mfi, 2)
  p_max = i

In [7]:
# Démarrage lorsque tous les MFI sont renseignés
df = df.loc[(p_max-1):]
df.reset_index(drop=True, inplace=True)

In [8]:
df.head(1)

,Date,Open,High,Low,Close,Volume,MFI_5,MFI_6,MFI_7,MFI_8,...,MFI_41,MFI_42,MFI_43,MFI_44,MFI_45,MFI_46,MFI_47,MFI_48,MFI_49,MFI_50
0,2006-12-08,18.81,19.27,18.81,19.27,850,29.69,25.11,20.03,40.0,...,58.55,60.0,59.23,59.79,60.67,61.3,61.77,61.77,61.73,61.73


In [9]:
df.shape

(3923, 52)

In [10]:
df.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
MFI_5     0
MFI_6     0
MFI_7     0
MFI_8     0
MFI_9     0
MFI_10    0
MFI_11    0
MFI_12    0
MFI_13    0
MFI_14    0
MFI_15    0
MFI_16    0
MFI_17    0
MFI_18    0
MFI_19    0
MFI_20    0
MFI_21    0
MFI_22    0
MFI_23    0
MFI_24    0
MFI_25    0
MFI_26    0
MFI_27    0
MFI_28    0
MFI_29    0
MFI_30    0
MFI_31    0
MFI_32    0
MFI_33    0
MFI_34    0
MFI_35    0
MFI_36    0
MFI_37    0
MFI_38    0
MFI_39    0
MFI_40    0
MFI_41    0
MFI_42    0
MFI_43    0
MFI_44    0
MFI_45    0
MFI_46    0
MFI_47    0
MFI_48    0
MFI_49    0
MFI_50    0
dtype: int64

In [11]:
# Si besoin de filtrer sur une période plutôt que all time
'''
df = df_source.loc[(df_source['Date']>='2017-01-01')]
df.reset_index(drop=True, inplace=True)
df.head()
'''

"\ndf = df_source.loc[(df_source['Date']>='2017-01-01')]\ndf.reset_index(drop=True, inplace=True)\ndf.head()\n"

# Fichier local #

In [ ]:
df_source = pd.read_excel("mfi_datasets/MFI_NDQ_nov.xlsx")

In [ ]:
df = df_source.loc[(df_source['Date']>='2016-12-01')]
df.reset_index(drop=True, inplace=True)
df.tail()

,Date,Open,High,Low,Close,Volume,MFI5,MFI6,MFI7,MFI8,MFI9,MFI10,MFI11,MFI12,MFI13,MFI14,MFI15,MFI16,MFI17,MFI18,MFI19,MFI20,MFI21,MFI22,MFI23,MFI24,MFI25,MFI26,MFI27,MFI28,MFI29,MFI30
1286,2021-12-10,928.799988,945.500000,923.000000,928.299988,3307,70.720846,56.886282,44.718156,49.787879,43.384343,50.799003,44.182856,46.069013,48.797481,42.508721,45.827084,49.037448,50.454772,52.027818,53.549414,55.168626,56.002446,54.704014,53.085020,51.562109,49.771268,51.178630,53.291471,54.002578,54.758379,55.925855
1287,2021-12-13,948.400024,956.799988,925.400024,927.400024,4794,88.093739,75.045596,62.161153,50.177900,54.333317,47.902895,54.247560,47.616952,49.281621,51.702002,45.370608,48.375454,51.299032,52.594953,54.037012,55.435678,56.928167,57.698382,56.411044,54.802654,53.286434,51.499379,52.812833,54.789457,55.456010,56.165170
1288,2021-12-14,924.200012,925.500000,883.799988,885.799988,7771,57.471665,69.386606,61.028857,52.225658,43.498022,47.629089,42.614441,48.916339,43.459399,45.103871,47.511277,42.111070,45.078174,47.987714,49.284637,50.733068,52.143233,53.653801,54.435705,53.288394,51.850872,50.491561,48.884224,50.200015,52.187001,52.858910
1289,2021-12-15,904.099976,906.500000,886.200012,886.200012,3845,35.555337,50.189756,62.825197,55.894437,48.419459,40.825111,44.904072,40.419805,46.661051,41.670026,43.298640,45.689887,40.673932,43.616070,46.511158,47.804853,49.252029,50.663361,52.177800,52.962774,51.876092,50.512783,49.221851,47.693114,49.007582,50.995702
1290,2021-12-16,950.900024,960.000000,907.799988,907.799988,11157,52.321677,56.831003,63.929877,71.054366,64.798182,57.689946,50.052524,52.990141,48.407852,53.154395,48.095086,49.388654,51.302492,46.194461,48.625740,51.040244,52.126356,53.346615,54.542071,55.830859,56.501328,55.427597,54.075831,52.790979,51.263300,52.406416


# Début programme #

## Fonctions ##

In [12]:
def performance (me, entreeF, sortieV, df_extrait, col):
    '''Fonction de calcul de la performance 
    avec un seuil d'entrée fixe, et sortie variable'''

    # Définition des entrées sorties
    e = entreeF
    s = me + sortieV

    # Ajout de la colonne mfi_lag
    shift = df_extrait[col].shift(1)
    shift[0]=shift[1]
    df_extrait['mfi_lag']=shift

    # Filtre dans l'historique des points de sorties variables
    df_entries = df_extrait.loc[ (df_extrait[col]>=e) & (df_extrait['mfi_lag']<=(e-0.01)) ].copy()
    df_entries['move']='achat'
    df_exits = df_extrait.loc[ (df_extrait[col]<=s) & (df_extrait['mfi_lag']>=(s+0.01)) ].copy()
    # On ajoute une revente si on replonge dans la zone oversold.
    # Les doublons de vente (si on est déjà revenu de lazone overbought) seront éliminés par la suite, cf doublons
    #df_exits = df_extrait.loc[ ((df_extrait[col]<=s) & (df_extrait['mfi_lag']>=(s+0.01))) | (df_extrait[col]<=e) & (df_extrait['mfi_lag']>=(e+0.01)) ].copy()
    df_exits['move']='vente'

    # Concatenation des achats/ventes et tri par ordre chronologique
    df_perf = pd.concat([df_entries,df_exits])
    df_perf.sort_values(by='Date',inplace=True)
    df_perf.reset_index(drop=True, inplace=True)

    # Suppression des entrées-sorties doubles (type achat-achat-vente, ou achat-vente-vente)
    doubles = []
    for i in range (1,df_perf.shape[0]):
        if df_perf['move'][i] == df_perf['move'][i-1]:
            doubles.append(i)
        else:
            continue
    df_perf_clean = df_perf.drop(index=doubles)
    df_perf_clean.reset_index(drop=True, inplace=True)

    # Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
    if df_perf_clean.empty != True: 
        if df_perf_clean.iloc[0,4] == 'vente':
            df_perf_clean = df_perf_clean.drop(index=0)
            df_perf_clean.reset_index(drop=True, inplace=True)

    # Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
    # Obtenir toute la dernière ligne du dataframe origienl, puis concat au df_perf_clean = vente au jour J
    if df_perf_clean.empty != True: 
        if df_perf_clean.iloc[-1,4] == 'achat':
            df_last_row = df_extrait.iloc[-1:].copy()
            df_last_row['move']='vente'
            df_perf_clean = pd.concat([df_perf_clean,df_last_row])
            df_perf_clean.reset_index(drop=True, inplace=True)

    # Calcul de la performance
    perfE = [1000]
    j=0
    # S'assurer que le Dataframe a bien 1 entrée 1 sortie, et n'est pas vide (out of bounds)
    if df_perf_clean.shape[0]>=2:
        for i in range (1,df_perf_clean.shape[0],2):
            '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
            # Correction avec ajustement des frais de transaction
            var = (df_perf_clean['Close'][i]-df_perf_clean['Close'][i-1])/df_perf_clean['Close'][i-1] -0.011
            varE = perfE[j]*(1+var)
            perfE.append(varE)
            j+=1
    else:
        # Dans le cas où DF vide : on ajoute la valeur d'entrée pour surperf 0
        perfE.append(1000)
    '''Choix de la performance totale en %'''
    overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)
    
    return overallP

In [13]:
def es_variables (mediane,entreeV,sortieV,df_extrait,col):
    ''' Fonction de calcul de la performance
    seuils d'entrée et de sortie variables dans les plages définies en amont
    avec equilibre à la médiane '''

    matrice = []

    # Définition des entrées sorties
    for eV in entreeV:
        eF = mediane - eV
        ligne = [performance(mediane,eF,sV,df_extrait,col) for sV in sortieV]
        matrice.append(ligne)

    return matrice

In [19]:
def export_heatmap (matrice,base,pe,ps,titre):
    colonnes = []
    for i in ps:
        nom = "s="+str(base+i)
        colonnes.append(nom)

    ind = []
    for i in pe:
        nom = "e="+str(base-i)
        ind.append(nom)

    df_HM = pd.DataFrame(matrice,columns=colonnes)
    df_HM.set_index(pd.Index(ind), inplace=True)

    fig, ax = plt.subplots(figsize = (30, 15))
    sns.heatmap(df_HM, cmap ='RdYlGn', linewidths = 0.30, annot = True, fmt=".0f")
    ax.set_title(titre)
    chemin = "/content/drive/MyDrive/Colab Notebooks/performancesMFI/NDQ2x-allTime/NDX2x_AT_" + titre + "_avr29.png"
    plt.savefig(chemin)
    # Protection memoire, sinon ouverture de 30 plots
    plt.close(fig)

## Execution ##

In [15]:
#df = df_source
print("Buy & Hold Performance = {} %".format(round(((df['Close'].iloc[-1] - df['Close'].iloc[0])/df['Close'].iloc[0])*100,2)) )

Buy & Hold Performance = 3216.04 %


In [20]:
dict_res = {'MFI':[],'Meilleure Perf':[]}

# Pour l'ensemble des colonnes MFI entre 5 et 50
for i in range(5,51):

    # Selection du MFIp
    colonneMFI = "MFI_"+str(i)
    df_extract = df[['Date','Close',colonneMFI]].copy()

    # Obtention de la mediane
    med = round(df[colonneMFI].median())

    # Définition des plages d'entrée et sortie selon min-max de la colonne MFI sélectionnée
    # On selectionne df_stats qui a éliminé les MFI nuls, sinon ceux en lag à l'origine vont définir 0 en entrée min
    entree_min = math.ceil(df[colonneMFI].min())
    sortie_max = math.trunc(df[colonneMFI].max())
    delta_e = med - entree_min
    delta_s = sortie_max - med
    plage_entrees = [num for num in range(5,delta_e)]
    plage_sorties = [num for num in range(10,delta_s)]

    m = es_variables(med, plage_entrees, plage_sorties, df_extract, colonneMFI)
    tableau_perf = np.array(m)

    dict_res["MFI"].append(colonneMFI)
    dict_res["Meilleure Perf"].append(np.max(tableau_perf))

    export_heatmap(tableau_perf, med, plage_entrees, plage_sorties, colonneMFI)

df_res = pd.DataFrame(dict_res)
df_res.sort_values(by="Meilleure Perf", ascending=False)

,MFI,Meilleure Perf
12,MFI_17,7427.75
7,MFI_12,5718.95
9,MFI_14,5710.06
11,MFI_16,5287.74
6,MFI_11,4583.21
8,MFI_13,4526.18
10,MFI_15,4505.60
5,MFI_10,4022.19
31,MFI_36,3931.43
32,MFI_37,3821.72


In [21]:
df_res.to_excel("/content/drive/MyDrive/Colab Notebooks/performancesMFI/NDQ2x-allTime/NDX2x_04-29.xlsx", sheet_name='all_time')

# Verification de la performance #

## Vérification unitaire ##

In [22]:
periode = 17
entree = 38
sortie = 87

In [23]:
df_best = df[['Date','Close','MFI_'+str(periode)]]

In [24]:
shift = df_best['MFI_'+str(periode)].shift(1)
shift[0]=shift[1]
df_best['mfi_lag']=shift

# Filtre dans l'historique des points d'entrée / sorties
df_entries = df_best.loc[ (df_best['MFI_'+str(periode)]>=entree) & (df_best['mfi_lag']<=(entree-0.01)) ]
df_entries['move']='achat'
df_exits = df_best.loc[ (df_best['MFI_'+str(periode)]<=sortie) & (df_best['mfi_lag']>=(sortie+0.01)) ]
# On ajoute une revente si on replonge dans la zone oversold.
# Les doublons de vente (si on est déjà revenu de lazone overbought) seront éliminés par la suite, cf doublons
#df_exits = df_best.loc[ ((df_best['MFI'+str(periode)]<=sortie) & (df_best['mfi_lag']>=(sortie+0.01))) | (df_best['MFI'+str(periode)]<=entree) & (df_best['mfi_lag']>=(entree+0.01)) ].copy()
df_exits['move']='vente'

# Concatenation des achats/ventes et tri par ordre chronologique
df_perf = pd.concat([df_entries,df_exits])
df_perf.sort_values(by='Date',inplace=True)
df_perf.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [25]:
df_perf

,Date,Close,MFI_17,mfi_lag,move
0,2006-12-12,19.22,41.15,35.70,achat
1,2007-01-12,20.62,42.52,37.56,achat
2,2007-02-23,20.16,41.35,37.78,achat
3,2007-03-01,18.27,41.52,28.36,achat
4,2007-03-08,18.34,46.28,35.00,achat
...,...,...,...,...,...
136,2022-02-01,785.20,41.11,34.63,achat
137,2022-02-15,734.90,44.67,37.80,achat
138,2022-03-01,701.90,41.63,37.38,achat
139,2022-03-09,657.70,42.92,34.94,achat


In [26]:
df_perf.to_excel('/content/drive/MyDrive/NDQ_AT_raw_17-38-87.xlsx', header=True, index=False)

In [27]:
doubles = []
for i in range (1,df_perf.shape[0]):
    if df_perf['move'][i] == df_perf['move'][i-1]:
        doubles.append(i)
    else:
        continue
    
df_perf_clean = df_perf.drop(index=doubles)
df_perf_clean.reset_index(drop=True, inplace=True)

In [28]:
# Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
if df_perf_clean['move'][0] == 'vente':
    df_perf_clean = df_perf_clean.drop(index=0)
    df_perf_clean.reset_index(drop=True, inplace=True)

In [29]:
# Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
# Obtenir toute la dernière ligne sous mf25, puis concat au df_perf_clean
if df_perf_clean.iloc[-1,4] == 'achat':
    df_last_row = df_best.iloc[-1:].copy()
    df_last_row['move']='vente'
    df_perf_clean = pd.concat([df_perf_clean,df_last_row])
    df_perf_clean.reset_index(drop=True, inplace=True)

In [30]:
df_perf_clean

,Date,Close,MFI_17,mfi_lag,move
0,2006-12-12,19.22,41.15,35.70,achat
1,2007-05-08,20.11,72.95,92.66,vente
2,2007-06-28,21.17,68.96,19.99,achat
3,2007-12-12,22.30,78.49,87.67,vente
4,2008-01-29,15.69,43.39,34.84,achat
5,2009-06-08,10.59,67.39,90.05,vente
6,2009-07-09,9.64,38.84,35.11,achat
7,2009-08-03,12.17,84.63,92.65,vente
8,2009-11-09,13.78,38.77,31.45,achat
9,2010-03-26,18.82,84.82,87.89,vente


In [ ]:
df_perf_clean.to_excel('/content/drive/MyDrive/NDQ_AT_17-38-87.xlsx', header=True, index=False)

In [32]:
perfE = [1000]
j=0
for i in range (1,df_perf_clean.shape[0],2):
    '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
    var = (df_perf_clean['Close'][i]-df_perf_clean['Close'][i-1])/df_perf_clean['Close'][i-1] -0.011
    varE = perfE[j]*(1+var)
    perfE.append(varE)
    j+=1
# Performance totale en % 
overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)
print("Peformance Totale : ",overallP)

Peformance Totale :  7427.75


## Vérification avec contrôle Nb trades minimal ##

In [ ]:
d_best_performers = {
    "MFI" : [17,16,16,15,14,13,12,7],
    "entree" : [38,5,38,33,35,29,31,19],
    "sortie" : [87,99,87,92,92,92,92,99]
}

In [ ]:
for i in range(0,len(d_best_performers["MFI"])):
  
  mfi = d_best_performers["MFI"][i]
  entree = d_best_performers["entree"][i]
  sortie = d_best_performers["sortie"][i]

  # Selection du MFIp
  colonneMFI = "MFI_"+str(mfi)
  df_extrait = df[['Date','Close',colonneMFI]].copy()
  
  # Ajout de la colonne mfi_lag
  shift = df_extrait[colonneMFI].shift(1)
  shift[0]=shift[1]
  df_extrait['mfi_lag']=shift

  # Filtre dans l'historique des points de sorties variables
  df_entries = df_extrait.loc[ (df_extrait[colonneMFI] >= entree) & (df_extrait['mfi_lag'] <= (entree-0.01)) ].copy()
  df_entries['move']='achat'
  df_exits = df_extrait.loc[ (df_extrait[colonneMFI] <= sortie) & (df_extrait['mfi_lag'] >= (sortie+0.01)) ].copy()
  # On ajoute une revente si on replonge dans la zone oversold.
  # Les doublons de vente (si on est déjà revenu de lazone overbought) seront éliminés par la suite, cf doublons
  #df_exits = df_extrait.loc[ ((df_extrait[col]<=s) & (df_extrait['mfi_lag']>=(s+0.01))) | (df_extrait[col]<=e) & (df_extrait['mfi_lag']>=(e+0.01)) ].copy()
  df_exits['move']='vente'

  # Concatenation des achats/ventes et tri par ordre chronologique
  df_perf = pd.concat([df_entries,df_exits])
  df_perf.sort_values(by='Date',inplace=True)
  df_perf.reset_index(drop=True, inplace=True)

  # Suppression des entrées-sorties doubles (type achat-achat-vente, ou achat-vente-vente)
  doubles = []
  for i in range (1,df_perf.shape[0]):
      if df_perf['move'][i] == df_perf['move'][i-1]:
          doubles.append(i)
      else:
          continue
  df_perf_clean = df_perf.drop(index=doubles)
  df_perf_clean.reset_index(drop=True, inplace=True)

  nb_trades = df_perf_clean.shape[0]

  # Suppression de la 1ere ligne si vente, mais après nettoyage des doublons
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[0,4] == 'vente':
          df_perf_clean = df_perf_clean.drop(index=0)
          df_perf_clean.reset_index(drop=True, inplace=True)

  # Si dernière ligne est achat non soldé -> insérer la dernière ligne en vente
  # Obtenir toute la dernière ligne du dataframe origienl, puis concat au df_perf_clean = vente au jour J
  if df_perf_clean.empty != True: 
      if df_perf_clean.iloc[-1,4] == 'achat':
          df_last_row = df_extrait.iloc[-1:].copy()
          df_last_row['move']='vente'
          df_perf_clean = pd.concat([df_perf_clean,df_last_row])
          df_perf_clean.reset_index(drop=True, inplace=True)

  # Calcul de la performance
  perfE = [1000]
  j=0
  # S'assurer que le Dataframe a bien 1 entrée 1 sortie, et n'est pas vide (out of bounds)
  if df_perf_clean.shape[0]>=2:
      for i in range (1,df_perf_clean.shape[0],2):
          '''par step 2 pour ne traiter que les lignes impaires correspondant aux lignes de vente'''
          var = (df_perf_clean['Close'][i]-df_perf_clean['Close'][i-1])/df_perf_clean['Close'][i-1]
          varE = perfE[j]*(1+var)
          perfE.append(varE)
          j+=1
  else:
      # Dans le cas où DF vide : on ajoute la valeur d'entrée pour surperf 0
      perfE.append(1000)
  '''Choix de la performance totale en %'''
  overallP = round(((perfE[-1]-perfE[0])/perfE[0])*100,2)
  
  config = str(mfi) + "_e" + str(entree) + "s_" + str(sortie)

  print("Paramètres : {} , Performance : {} , Nombre de trades : {}".format(config,overallP,nb_trades) )

Paramètres : 17_e38s_87 , Performance : 8955.44 , Nombre de trades : 31
Paramètres : 16_e5s_99 , Performance : 5549.31 , Nombre de trades : 3
Paramètres : 16_e38s_87 , Performance : 4814.81 , Nombre de trades : 37
Paramètres : 15_e33s_92 , Performance : 5160.35 , Nombre de trades : 21
Paramètres : 14_e35s_92 , Performance : 6596.86 , Nombre de trades : 23
Paramètres : 13_e29s_92 , Performance : 5148.7 , Nombre de trades : 27
Paramètres : 12_e31s_92 , Performance : 6027.67 , Nombre de trades : 33
Paramètres : 7_e19s_99 , Performance : 4668.56 , Nombre de trades : 69


# Détails du code #

In [ ]:
df_stats

,Date,Close,mfi25,mfi_lag
0,2017-09-01,4892,60.94,60.94
1,2017-09-02,4578,56.82,60.94
2,2017-09-03,4582,55.93,56.82
3,2017-09-04,4236,52.24,55.93
4,2017-09-05,4376,48.62,52.24
...,...,...,...,...
1464,2021-09-04,49944,62.60,62.28
1465,2021-09-05,51753,62.58,62.60
1466,2021-09-06,52633,66.74,62.58
1467,2021-09-07,46811,60.72,66.74


In [ ]:
df25 = df[['Date','Close','mfi25']]

In [ ]:
# Obtention de la mediane
null_mfi = df25.loc[(df25['mfi25']==0)].index
df_stats = df25.drop(null_mfi).copy()
med = round(df_stats['mfi25'].median())
print("Mediane : ",med)

Mediane :  53


In [ ]:
entree_min = df_stats['mfi25'].min()
sortie_max = df_stats['mfi25'].max()
print("entrée min : {}\nsortie max : {}".format(entree_min, sortie_max))
print("entrée : {}\nsortie : {}".format(math.ceil(entree_min),math.trunc(sortie_max)))

entrée min : 18.92
sortie max : 92.68
entrée : 19
sortie : 92


In [ ]:
""" Définition des plages d'entrées et sorties
La plage d'entrée doit être de 10 à l'arrondi supérieur du mfi minimum
La plage de sortie doit être de 10 à l'arrondi inférieur du mfi maximum
Ce afin de garantir au moins une entrée et une sortie sur la durée d'analyse"""

entree_min = math.ceil(df_stats['mfi25'].min())
sortie_max = math.trunc(df_stats['mfi25'].max())
delta_e = med - entree_min
delta_s = sortie_max - med
plage_entrees = [num for num in range(10,delta_e)]
plage_sorties = [num for num in range(10,delta_s)]
print("delta entrees [10;{}]\ndelta sorties [10;{}]".format(delta_e, delta_s))

delta entrees [10;34]
delta sorties [10;39]


In [ ]:
liste_perfs =[]
for sortieV in plage_sorties:
    liste_perfs.append(performance(med, 43, sortieV, df25, 'mfi25'))

/var/folders/z_/8s6_kwjj6jddrmx0rfcr21bm0000gn/T/ipykernel_942/2111376690.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extrait['mfi_lag']=shift


In [ ]:
liste_perfs

[-54.24,
 -22.88,
 5.98,
 15.27,
 32.16,
 93.52,
 43.18,
 2.8,
 -2.21,
 -3.4,
 -6.98,
 120.11,
 72.64,
 156.14,
 165.1,
 858.17,
 709.29,
 709.29,
 681.57,
 681.57,
 681.57,
 681.57,
 792.56,
 772.68,
 627.41,
 627.41,
 515.77,
 515.77,
 515.77]